**Note: This is my EDA.  It's not pretty or nice and probably has some mistakes along the way.  Check out the associated `.py` file for a linted/formatted version of the solution.**

In [1]:
from datetime import datetime
from pathlib import Path
import sys

import polars as pl

# Gross hack to get jupyter to find hod stuff.
sys.path.append(str(Path(Path.cwd().parent)))
from hod.utils import get_data

# The Early Bird

(JAMES: Emphasis below mine when I think something will be useful.)

REF: https://hanukkah.bluebird.sh/5784/4/

The investigator called the phone number you found and left a message, and a man soon called back:

“Wow, that was years ago! It was quite an elegant tapestry.

“It took a lot of patience, but I did manage to get the dirt out of one section, which uncovered a superb owl. I put it up on my wall, and sometimes at night I swear I could hear the owl hooting.

“A few weeks later my bike chain broke on the way home, and I needed to get it fixed before work the next day. Thankfully, this woman I met on Tinder came over at 5am with her bike chain repair kit and some pastries from Noah’s. Apparently she liked to get up before dawn and claim the **first pastries that came out of the oven**.

“I didn’t have any money or I would’ve paid her for her trouble. She really liked the tapestry, though, so I wound up giving it to her.

“I don’t remember her name or anything else about her.”

Can you find the bicycle fixer’s phone number?

In [ ]:
# > 2017


In [25]:
data = get_data(data_path=Path('./../data/5784/'))
df_customers = data["customers"]
df_orders = data["orders"]
df_orders_items = data["orders_items"]
df_products = data["products"]

df_pastries = (
    df_products
    .select(
        pl.col(["sku", "desc"])
    )
    .filter((
        pl.col("sku").str.contains("BKY"),
        ~pl.col("desc").str.contains("Bagel")
    ))
)

df_orders_containing_pastries = df_orders_items.join(df_pastries, on="sku", how="inner").select(pl.col("orderid")).unique()

df_pastry_order_data = df_orders.join(df_orders_containing_pastries, on="orderid", how="inner").select([pl.col(["orderid", "customerid", "ordered"])])

df_early_morning_pastry_orderers = df_pastry_order_data.filter((
    (pl.col("ordered").dt.hour() >= 0) &
    (pl.col("ordered").dt.hour() < 5)
))

customerid = df_early_morning_pastry_orderers.group_by(by="customerid").agg(pl.col("orderid").count()).sort(by="orderid", descending=True).item(row=0, column="customerid")

df_customers.filter(pl.col("customerid") == customerid).item(row=0, column="phone")


'607-231-3605'

customerid,name,address,citystatezip,birthdate,phone,timezone,lat,long
i64,str,str,str,date,str,str,f64,f64
2749,"""Renee Harmon""","""7A Nassau Ave""","""Brooklyn, NY 1…",1999-01-14,"""607-231-3605""","""America/New_Yo…",40.72347,-73.95097
